In [20]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import time
from sklearn.model_selection import train_test_split
from customDataset import DrowsyDataset as DD
from customModel import CustomCNN

In [21]:

# Define transforms to be applied on the images
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [22]:
# Create instances of the custom dataset class for train, validation and test data
dataset = DD('data/train/', data_transforms)

# # Get the labels from the dataset
# labels = np.array([label for _, label in dataset])

# Split the dataset into train and test sets, stratified by the target variable
train_data, test_data = train_test_split(dataset, test_size=0.2,  random_state=42)

# Print the sizes of train and test sets
print("Train set size:", len(train_data))
print("Test set size:", len(test_data))



Train set size: 2320
Test set size: 580


In [23]:
# Define batch size for DataLoader
batch_size = 32

test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [24]:
import os

# Find model files in the models directory
model_dir = 'models'
model_paths = [os.path.join(model_dir, file) for file in os.listdir(model_dir) if file.endswith('.pth')]

# Load the models
models = []
for model_path in model_paths:
    model = torch.load(model_path)
    model.eval()
    models.append(model)
models

[AlexNet(
   (features): Sequential(
     (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
     (1): ReLU(inplace=True)
     (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
     (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
     (4): ReLU(inplace=True)
     (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
     (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (7): ReLU(inplace=True)
     (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (9): ReLU(inplace=True)
     (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (11): ReLU(inplace=True)
     (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
   )
   (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
   (classifier): Sequential(
     (0): Dropout(p=0.5, inplace=False)
     (1): Linear(in_features=9216, out_feature

In [25]:


# Initialize lists to store the predicted labels and true labels
predicted_labels = []
true_labels = []

# Iterate over the test dataset and make predictions with each model
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    
    # Initialize lists to store predictions from each model
    model_predictions = []
    
    # Make predictions with each model
    for model in models:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        model_predictions.append(predicted.cpu().numpy())
    
    # Compute average ensemble predictions
    ensemble_predictions = sum(model_predictions) / len(models)
    
    # Append the ensemble predictions and true labels to the lists
    predicted_labels.extend(ensemble_predictions)
    true_labels.extend(labels.cpu().numpy())




In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


#  Compute the test metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print the test metrics
print("Accuracy:", round(accuracy,3))
print("Precision:", round(precision,3))
print("Recall:", round(recall,3))
print("F1 Score:", round(f1,3))

Accuracy: 0.907
Precision: 0.918
Recall: 0.907
F1 Score: 0.907
